In [221]:
import cirq
import numpy as np
from math import pi
#from cirq.ops import gate_operation
from cirq.value import Duration

#libraries to plot histogram
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

#google
from cirq import circuits, ops, sim, study, protocols, optimizers
from cirq.google import convert_to_xmon_gates
from cirq import circuits, ops, sim, study, protocols
from cirq.google import xmon_device

shots=1000

In [222]:
# define the length of the grid.
length = 5

#number of qubits used
N = 5

# define qubits on the grid.
#qubits = [cirq.GridQubit(i, j) for i in range(1) for j in range(length)] #per circuitXMON solo!

qubits = cirq.LineQubit.range(N)

print(qubits)

[cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4)]


In [223]:
def circuit(j,string):  
       
    circuit = cirq.Circuit()            
    
    #if(string == 'Noise'):
        #circuit.append(cirq.DepolarizingChannel(0.1).on_each(*qubits))
    
    m = 500 #number of application of amplitude damping
    gamma = 0.001 #parameters gamma of amplitude damping
    
    circuit.append(cirq.H(qubits[0]))

    for i in range(N-1):    
        circuit.append(cirq.CNOT(qubits[i],qubits[i+1]))
        if(string == 'Noise'): 
            if (i==0): 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i+1]))
            if (i==1): 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i+1])) 
            if (i==2): 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i+1]))
            if (i==3): 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i+1]))
        
    #print(circuit)    
    
    #if(string == 'Noise'):
       # for i in range(N-1):  
          # circuit.append(cirq.AmplitudeDampingChannel(0.1).on_each(*qubits)) 
        #circuit.append(cirq.DepolarizingChannel(0.07).on_each(*qubits))
        
    #insert U gate on all qubits
    phi = (pi*j)/(N+1)
    #A  unitary 1-qubit gate U 
    matrix=np.array([[1, 0], [0, np.exp(1.0j*phi)]])
    # matrix=np.array([[np.exp(-1.0j*phi), 1], [1, np.exp(1.0j*phi)]])
    moment_0 = cirq.Moment( cirq.SingleQubitMatrixGate(matrix).on(qubits[q]) for q in range(N) ) 
    # moment_0 = cirq.Moment( cirq.PhasedXPowGate().on(qubits[q]) for q in range(N) ) 
    circuit.append(moment_0)
    
    
    for i in reversed(range(N-1)):
        circuit.append(cirq.CNOT(qubits[i],qubits[i+1]))
        if(string == 'Noise'): 
            if (i==0): 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i+1]))
            if (i==1): 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i+1])) 
            if (i==2): 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i+1]))
            if (i==3): 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[i+1]))
                
    circuit.append(cirq.H(qubits[0]))
        
    #measurement
    circuit.append(cirq.measure(*qubits, key='x'))      

    #print(circuit)
        
    return circuit

In [213]:
def simulation(circuit,string):

    if(string == 'NoNoise'):
            
        f = open("counts_NoNoise.txt","a+")
        simulator = cirq.Simulator()
        results = simulator.run(circuit, repetitions=shots)
        #results = simulator.simulate(circuit)
        #print(results.dirac_notation())
        h = results.histogram(key='x') 
        #print(results)
        print(1/shots*h[0])

        #f.write('{}'.format(1/shots*h[0]))
        #f.write(',')
        f.close()  
        
    if(string == 'Noise'):
        
        f = open("counts_Noise.txt","a+")
    #Density simulator
        simulator = cirq.DensityMatrixSimulator()           
        #results = simulator.run(circuit, repetitions=shots)
        results = simulator.simulate(circuit,qubit_order = [qubits[0],qubits[1],qubits[2],qubits[3],qubits[4]]) #per accedere alla matrice densità
        g = results.final_density_matrix
        print(np.around(g, 3))  #per accedere alla matrice densità
        print('\n')
        print(g.shape)
        print(g.ndim)
        print('\n')
        l = cirq.partial_trace(g, keep_indices = [0])
        print(l)
        #print(results)
        #cirq.partial_trace(results,[1,2,3])
        f.write('{}'.format(g))
        f.write(',')
        f.close()  
        
    #Generic simulator
        ##results = cirq.sample(circuit, repetitions=shots)
        #simulator = cirq.DensityMatrixSimulator()           
        #results = simulator.run(circuit, repetitions=shots)
        #h = results.histogram(key='x') 
        #print(results)
        #print(1/shots*h[0])

        #f.write('{}'.format(1/shots*h[0]))
        #f.write(',')
        #f.close()  
        

In [224]:
def main ():    

    for j in range(2*N+1+1):   # for j in range(2*N+1):
        print('Histogram j = {}'.format(j))
        #simulation(circuit(j, 'NoNoise'), 'NoNoise')
        simulation(circuit(j, 'Noise'), 'Noise')
        

In [226]:
if __name__ == '__main__':
    main()

Histogram j = 0
[[0.626+0.j 0.   +0.j 0.   +0.j ... 0.   +0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.026+0.j 0.   +0.j ... 0.   +0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.021+0.j ... 0.   +0.j 0.   +0.j 0.   +0.j]
 ...
 [0.   +0.j 0.   +0.j 0.   +0.j ... 0.002+0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j ... 0.   +0.j 0.004+0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j ... 0.   +0.j 0.   +0.j 0.002+0.j]]


(32, 32)
2


[[0.6264695 +0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.        +0.j]
 [0.        +0.j 0.02576634+0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.02084494+0.j ... 0.        +0.j
  0.        +0.j 0.        +0.j]
 ...
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.00203056+0.j
  0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.00379716+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.        

In [225]:
def circuit(j,string):  
       
    circuit = cirq.Circuit()            
    
    #if(string == 'Noise'):
        #circuit.append(cirq.DepolarizingChannel(0.1).on_each(*qubits))
    
    pb = 0.01
    pp = 0.01
    p = 0.03
    gamma = 0.1 #parameters gamma of amplitude damping
    m = 1 #number of application of amplitude damping
    
    circuit.append(cirq.bit_flip(pb).on(qubits[0]))
    circuit.append(cirq.H(qubits[0]))
    circuit.append(cirq.CNOT(qubits[0],qubits[1]))
    circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[1]))  
    circuit.append(cirq.CNOT(qubits[1],qubits[2]))    
    circuit.append(cirq.DepolarizingChannel(p).on(qubits[2]))
    
    circuit.append(cirq.phase_flip(pp).on(qubits[3]))
    
    circuit.append(cirq.CNOT(qubits[2],qubits[3]))   
    circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[2]))  
    circuit.append(cirq.bit_flip(pb).on(qubits[3]))
    circuit.append(cirq.CNOT(qubits[3],qubits[4]))     
    circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[4]))     
        
    #insert U gate on all qubits
    phi = (pi*j)/(N+1)
    #A  unitary 1-qubit gate U 
    matrix=np.array([[1, 0], [0, np.exp(1.0j*phi)]])
    # matrix=np.array([[np.exp(-1.0j*phi), 1], [1, np.exp(1.0j*phi)]])
    moment_0 = cirq.Moment( cirq.SingleQubitMatrixGate(matrix).on(qubits[q]) for q in range(N) ) 
    # moment_0 = cirq.Moment( cirq.PhasedXPowGate().on(qubits[q]) for q in range(N) ) 
    circuit.append(moment_0)

    circuit.append(cirq.DepolarizingChannel(p).on(qubits[4]))    
    circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[3]))    
    circuit.append(cirq.CNOT(qubits[3],qubits[4])) 
    circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[3]))    
    circuit.append(cirq.bit_flip(pb).on(qubits[2]))
    
    circuit.append(cirq.CNOT(qubits[2],qubits[3])) 
    circuit.append(cirq.phase_flip(pp).on(qubits[2]))
    circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[1]))     
    circuit.append(cirq.AmplitudeDampingChannel(gamma).on(qubits[0])) 
    
    circuit.append(cirq.CNOT(qubits[1],qubits[2]))     
    
    circuit.append(cirq.DepolarizingChannel(p).on(qubits[1]))     
    circuit.append(cirq.CNOT(qubits[0],qubits[1]))     
    circuit.append(cirq.bit_flip(pb).on(qubits[0]))      
    circuit.append(cirq.bit_flip(pb).on(qubits[4]))  
    circuit.append(cirq.H(qubits[0]))
        
    #measurement
    #circuit.append(cirq.measure(*qubits, key='x'))      

    #print(circuit)
        
    return circuit



In [124]:
def circuit(j,string):  
       
    circuit = cirq.Circuit()            
    
    #if(string == 'Noise'):
        #circuit.append(cirq.DepolarizingChannel(0.1).on_each(*qubits))
    
    m = 30 #number of application of amplitude damping
    gamma = 0.001 #parameters gamma of amplitude damping
    
    circuit.append(cirq.H(qubits[0]))

    for i in range(N-1):    
        circuit.append(cirq.CNOT(qubits[i],qubits[i+1]))
        if(string == 'Noise'): 
            if (i==0): 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i+1]))
            if (i==1): 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i+1])) 
            if (i==2): 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i+1]))
            if (i==3): 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i+1]))
        
    #print(circuit)    
    
    #if(string == 'Noise'):
       # for i in range(N-1):  
          # circuit.append(cirq.AmplitudeDampingChannel(0.1).on_each(*qubits)) 
        #circuit.append(cirq.DepolarizingChannel(0.07).on_each(*qubits))
        
    #insert U gate on all qubits
    phi = (pi*j)/(N+1)
    #A  unitary 1-qubit gate U 
    matrix=np.array([[1, 0], [0, np.exp(1.0j*phi)]])
    # matrix=np.array([[np.exp(-1.0j*phi), 1], [1, np.exp(1.0j*phi)]])
    moment_0 = cirq.Moment( cirq.SingleQubitMatrixGate(matrix).on(qubits[q]) for q in range(N) ) 
    # moment_0 = cirq.Moment( cirq.PhasedXPowGate().on(qubits[q]) for q in range(N) ) 
    circuit.append(moment_0)
    
    
    for i in reversed(range(N-1)):
        circuit.append(cirq.CNOT(qubits[i],qubits[i+1]))
        if(string == 'Noise'): 
            if (i==0): 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i+1]))
            if (i==1): 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i+1])) 
            if (i==2): 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i+1]))
            if (i==3): 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i])) 
                for k in range(m): circuit.append(cirq.DepolarizingChannel(gamma).on(qubits[i+1]))
                
    circuit.append(cirq.H(qubits[0]))
        
    #measurement
    circuit.append(cirq.measure(*qubits, key='x'))      

    #print(circuit)
        
    return circuit


In [ ]:
circuit2 = cirq.Circuit()            
    
#circuit2.append(cirq.H(qubits[0]))
#circuit2.append(cirq.CNOT(qubits[0],qubits[1]))
#circuit2.append(cirq.CNOT(qubits[1],qubits[2]))
#circuit2.append(cirq.CNOT(qubits[2],qubits[3]))
#circuit2.append(cirq.CNOT(qubits[3],qubits[4]))

for i in range(N-1):    
    circuit2.append(cirq.CNOT(qubits[i],qubits[i+1]))
    if (i==0): 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i])) 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i+1]))
    if (i==1): 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i])) 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i+1])) 
    if (i==2): 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i])) 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i+1]))
    if (i==3): 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i])) 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i+1]))          

for i in reversed(range(N-1)):
    circuit2.append(cirq.CNOT(qubits[i],qubits[i+1]))
    if (i==0): 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i])) 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i+1]))
    if (i==1): 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i])) 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i+1])) 
    if (i==2): 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i])) 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i+1]))
    if (i==3): 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i])) 
        circuit2.append(cirq.AmplitudeDampingChannel(0.1).on(qubits[i+1]))          

    
print(circuit2)
        


In [269]:
qubits = cirq.LineQubit.range(2)
circuit = cirq.Circuit() 
circuit.append(cirq.H(qubits[0]))
circuit.append(cirq.CNOT(qubits[0],qubits[1]))
#circuit.append(cirq.bit_flip(0.1).on(qubits[0]))
#circuit.append(cirq.H(qubits[1]))

simulator = cirq.DensityMatrixSimulator()
result = simulator.simulate(circuit,qubit_order = [qubits[0],qubits[1]])
g = result.final_density_matrix
#g = result.density_matrix()
print(np.around(g, 3))


#print(g.shape)
#print(g.ndim)
print('\n')
l = cirq.partial_trace(g, keep_indices = [0])
#print(l)

a = np.array([[[[1,2],[4,5]],[[1,2],[4,5]]],[[[1,2],[4,5]],[[1,2],[4,5]]]])
print(a.shape)
print(a.ndim)
l = cirq.partial_trace(a, keep_indices = [0])
print(l)

# prints
# [[0.6  +0.j 0.447+0.j]
#  [0.447+0.j 0.4  +0.j]]

[[0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]]


(2, 2, 2, 2)
4
[[3 9]
 [3 9]]
